In [ ]:
# ONS (Census 2011) statistics for Lewisham.
# Fetches Nomis data releases and prepares them for use in Lewisham ward profiles.
#
# This requires the OA->LSOA lookup table produced by "lbl_boundaries_oa.ipynb"
# as well as the OA->WD22 lookup table produced by "lbl_boundaries_2022_wards.ipynb"

In [ ]:
import pandas as pd

from google.colab import drive
import google.colab.files as files

import IPython

Tools
==
Helpers to reformat & aggregate the Nomis source data.

In [ ]:
# Extract the subset we need.
def filter_nomis_data(d, date_values, geography_values):
  return d[d.date.isin(date_values) & 
           d.geography.isin(geography_values)]

# Reformat Nomis data into a lookup table format:
# clear labelling of the index (geography) column, 
# remove superfluous columns.
def format_nomis_data(d, geo_colname):
  return d.\
           rename(columns={'geography code': geo_colname}).\
           drop(columns=['date', 'geography'])

# Convenience function to call the above.
def nomis_to_oa(d, years, oa_list):
  return format_nomis_data(filter_nomis_data(d, years, oa_list), geo_colname='OA11CD')

In [ ]:
# Aggregate OA-level data to LSOAs, Wards, etc. (Calculates the sum.)
def aggregate_oa_groups(oa_data, oa_group_table, oa_col, group_col):
  return pd.merge(oa_group_table, oa_data, on=oa_col, how='right').\
              drop(columns=[oa_col]).\
              groupby(group_col).sum()

Data
==
GDrive mount
--

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
oa_lookups_dir = f"{project_dir}/lookups/oa"
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
output_dir = f"{project_dir}/country_of_birth"

In [ ]:
!mkdir -p '{output_dir}'

Downloads
--

In [ ]:
# Country of Birth (detailed), OAs in London 2011
# Source: 
# https://www.nomisweb.co.uk/census/2011/QS203EW
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_524_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299' \
  -O "{output_dir}/census11_country_of_birth_oa11_london.csv"

--2021-11-29 12:02:40--  https://www.nomisweb.co.uk/api/v01/dataset/nm_524_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.3.145
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.3.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/country_of_birth/census11_country_of_birth_detailed_oa11_london.csv’

/content/gdrive/MyD     [                 <=>]   5.10M  1.25MB/s    in 21s     

2021-11-29 12:03:04 (249 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/country_of_birth/census11_country_of_birth_detailed_oa11_london.csv’ saved [5348833]



In [ ]:
# For certain variables: get reference data for other geographies.
#
# Nomis geography codes are derived from their GEOGRAPHY and GEOGRAPHY_TYPECODE 
# fields, both numeric. For our purposes they are as follows:
# England: 2092957699TYPE499
# London: 2013265927TYPE480
# Lewisham: 1946157254TYPE464

# And optionally:
# United Kingdom: 2092957697TYPE499 -- this also returns nation-level records
# Great Britain: 2092957698TYPE499
# England and Wales: 2092957703TYPE499

# NOTE that not all variables are available at all levels of aggregation.

In [ ]:
# Country of Birth, OAs in London 2011
# Source: 
# https://www.nomisweb.co.uk/census/2011/QS203EW
!wget 'https://www.nomisweb.co.uk/api/v01/dataset/nm_524_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699TYPE499,2013265927TYPE480,1946157254TYPE464' \
  -O "{output_dir}/census11_country_of_birth_references.csv"

--2021-11-29 12:04:04--  https://www.nomisweb.co.uk/api/v01/dataset/nm_524_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2092957699TYPE499,2013265927TYPE480,1946157254TYPE464
Resolving www.nomisweb.co.uk (www.nomisweb.co.uk)... 129.234.3.145
Connecting to www.nomisweb.co.uk (www.nomisweb.co.uk)|129.234.3.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/country_of_birth/census11_country_of_birth_references.csv’

/content/gdrive/MyD     [ <=>                ]   8.06K  --.-KB/s    in 0.04s   

2021-11-29 12:04:05 (198 KB/s) - ‘/content/gdrive/MyDrive/WardProfiles/country_of_birth/census11_country_of_birth_references.csv’ saved [8253]



Lookups
--
Used to match up OAs with their LSOAs, and 2022 electoral wards

In [ ]:
# All OAs and their LSOAs in Lewisham
oa_lsoa_join = pd.read_csv(f"{oa_lookups_dir}/lbl_oa11_lsoa11_msoa11_lad20_rgn20_202012.csv")
oa_lsoa_join = oa_lsoa_join[['OA11CD', 'LSOA11CD']]
oa_lsoa_join.head()

,OA11CD,LSOA11CD
0,E00016277,E01003220
1,E00016278,E01003220
2,E00016285,E01003220
3,E00016257,E01003221
4,E00016263,E01003221


In [ ]:
# All OAs and their 2022 wards in Lewisham
oa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_oa11_wd22_proposed.csv")
oa_wd22_join = oa_wd22_join[['OA11CD', 'WD22CD_proposed']]
oa_wd22_join.head()

,OA11CD,WD22CD_proposed
0,E00016403,E05013721
1,E00016442,E05013721
2,E00016407,E05013721
3,E00016404,E05013721
4,E00016402,E05013721


Process & export
==
Lewisham
--

In [ ]:
# For filtering of source data
lbl_oa_list = oa_lsoa_join.OA11CD.unique() # All OAs in Lewisham
years = [2011] # Just a precaution, in case any of the downloads include multiple periods

for datname in ['country_of_birth']:
  IPython.display.display(f"=== {datname} ===")
  # Load the Nomis data
  d = pd.read_csv(f"{output_dir}/census11_{datname}_oa11_london.csv")

  # Simplify the column names
  d.columns = d.columns.str.replace(
      # Remove the redundant title prefix and qualifier suffix
      r'^.*?: (.*?); measures: Value', 
      r'\1')
  d = d.rename(columns={'All categories: Country of birth': 'Total'})

  # Drop unused columns
  if 'Rural Urban' in d.columns:
    d.drop(columns='Rural Urban', inplace=True) # Lewisham is all urban

  # Transform to OA index data
  lbl_oa = nomis_to_oa(d, years, lbl_oa_list)
  lbl_oa.to_csv(f"{output_dir}/lbl_{datname}_oa11.csv", index=False)

  # Aggregate to LSOA level
  lbl_lsoa = aggregate_oa_groups(lbl_oa, oa_lsoa_join, oa_col='OA11CD', group_col='LSOA11CD')
  lbl_lsoa.to_csv(f"{output_dir}/lbl_{datname}_lsoa11.csv", index=True)
  IPython.display.display(lbl_lsoa.head())

  # Aggregate to WD22 level
  lbl_wd22 = aggregate_oa_groups(lbl_oa, oa_wd22_join, oa_col='OA11CD', group_col='WD22CD_proposed')
  lbl_wd22.to_csv(f"{output_dir}/lbl_{datname}_wd22.csv", index=True)
  IPython.display.display(lbl_wd22.head())

'=== country_of_birth ==='

,Total,Europe: Total,Europe: United Kingdom: Total,Europe: United Kingdom: England,Europe: United Kingdom: Northern Ireland,Europe: United Kingdom: Scotland,Europe: United Kingdom: Wales,Europe: Great Britain not otherwise specified,Europe: United Kingdom not otherwise specified,Europe: Guernsey,Europe: Jersey,Europe: Channel Islands not otherwise specified,Europe: Isle of Man,Europe: Ireland,Europe: Other Europe: Total,Europe: Other Europe: EU Countries: Total,Europe: Other Europe: EU countries: Member countries in March 2001: Total,Europe: Other Europe: EU countries: Member countries in March 2001: France,Europe: Other Europe: EU countries: Member countries in March 2001: Germany,Europe: Other Europe: EU countries: Member countries in March 2001: Italy,Europe: Other Europe: EU countries: Member countries in March 2001: Portugal,Europe: Other Europe: EU countries: Member countries in March 2001: Spain (including Canary Islands),Europe: Other Europe: EU countries: Member countries in March 2001: Other member countries in March 2001,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Total,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Lithuania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Poland,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Romania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Other EU accession countries,Europe: Other Europe: Rest of Europe: Total,Europe: Other Europe: Rest of Europe: Turkey,Europe: Other Europe: Rest of Europe: Other Europe,Africa: Total,Africa: North Africa,Africa: Central and Western Africa: Total,Africa: Central and Western Africa: Ghana,Africa: Central and Western Africa: Nigeria,Africa: Central and Western Africa: Other Central and Western Africa,Africa: South and Eastern Africa: Total,Africa: South and Eastern Africa: Kenya,Africa: South and Eastern Africa: Somalia,Africa: South and Eastern Africa: South Africa,Africa: South and Eastern Africa: Zimbabwe,Africa: South and Eastern Africa: Other South and Eastern Africa,Africa: Africa not otherwise specified,Middle East and Asia: Total,Middle East and Asia: Middle East: Total,Middle East and Asia: Middle East: Iran,Middle East and Asia: Middle East: Other Middle East,Middle East and Asia: Eastern Asia: Total,Middle East and Asia: Eastern Asia: China,Middle East and Asia: Eastern Asia: Hong Kong (Special Administrative Region of China),Middle East and Asia: Eastern Asia: Other Eastern Asia,Middle East and Asia: Southern Asia: Total,Middle East and Asia: Southern Asia: Bangladesh,Middle East and Asia: Southern Asia: India,Middle East and Asia: Southern Asia: Pakistan,Middle East and Asia: Southern Asia: Sri Lanka,Middle East and Asia: Southern Asia: Other Southern Asia,Middle East and Asia: South-East Asia: Total,Middle East and Asia: South-East Asia: Philippines,Middle East and Asia: South-East Asia: Other South-East Asia,Middle East and Asia: Central Asia,The Americas and the Caribbean: Total,The Americas and the Caribbean: North America: Total,The Americas and the Caribbean: North America: United States,The Americas and the Caribbean: North America: Other North America,The Americas and the Caribbean: Central America,The Americas and the Caribbean: South America,The Americas and the Caribbean: The Caribbean: Total,The Americas and the Caribbean: The Caribbean: Jamaica,The Americas and the Caribbean: The Caribbean: Other Caribbean,Antarctica and Oceania: Total,Antarctica and Oceania: Antarctica,Antarctica and Oceania: Australasia: Total,Antarctica and Oceania: Australasia: Australia,Antarctica and Oceania: Australasia: Other Australasia,Antarctica and Oceania: Other Oceania,Other
LSOA11CD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E01003189,1670,1341,1208,1193,2,9,4,0,0,0,0,0,0,22,111,77,36,4,1,3,5,1,22,41,15,7,2,17,34,16,18,158,12,11

,Total,Europe: Total,Europe: United Kingdom: Total,Europe: United Kingdom: England,Europe: United Kingdom: Northern Ireland,Europe: United Kingdom: Scotland,Europe: United Kingdom: Wales,Europe: Great Britain not otherwise specified,Europe: United Kingdom not otherwise specified,Europe: Guernsey,Europe: Jersey,Europe: Channel Islands not otherwise specified,Europe: Isle of Man,Europe: Ireland,Europe: Other Europe: Total,Europe: Other Europe: EU Countries: Total,Europe: Other Europe: EU countries: Member countries in March 2001: Total,Europe: Other Europe: EU countries: Member countries in March 2001: France,Europe: Other Europe: EU countries: Member countries in March 2001: Germany,Europe: Other Europe: EU countries: Member countries in March 2001: Italy,Europe: Other Europe: EU countries: Member countries in March 2001: Portugal,Europe: Other Europe: EU countries: Member countries in March 2001: Spain (including Canary Islands),Europe: Other Europe: EU countries: Member countries in March 2001: Other member countries in March 2001,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Total,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Lithuania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Poland,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Romania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Other EU accession countries,Europe: Other Europe: Rest of Europe: Total,Europe: Other Europe: Rest of Europe: Turkey,Europe: Other Europe: Rest of Europe: Other Europe,Africa: Total,Africa: North Africa,Africa: Central and Western Africa: Total,Africa: Central and Western Africa: Ghana,Africa: Central and Western Africa: Nigeria,Africa: Central and Western Africa: Other Central and Western Africa,Africa: South and Eastern Africa: Total,Africa: South and Eastern Africa: Kenya,Africa: South and Eastern Africa: Somalia,Africa: South and Eastern Africa: South Africa,Africa: South and Eastern Africa: Zimbabwe,Africa: South and Eastern Africa: Other South and Eastern Africa,Africa: Africa not otherwise specified,Middle East and Asia: Total,Middle East and Asia: Middle East: Total,Middle East and Asia: Middle East: Iran,Middle East and Asia: Middle East: Other Middle East,Middle East and Asia: Eastern Asia: Total,Middle East and Asia: Eastern Asia: China,Middle East and Asia: Eastern Asia: Hong Kong (Special Administrative Region of China),Middle East and Asia: Eastern Asia: Other Eastern Asia,Middle East and Asia: Southern Asia: Total,Middle East and Asia: Southern Asia: Bangladesh,Middle East and Asia: Southern Asia: India,Middle East and Asia: Southern Asia: Pakistan,Middle East and Asia: Southern Asia: Sri Lanka,Middle East and Asia: Southern Asia: Other Southern Asia,Middle East and Asia: South-East Asia: Total,Middle East and Asia: South-East Asia: Philippines,Middle East and Asia: South-East Asia: Other South-East Asia,Middle East and Asia: Central Asia,The Americas and the Caribbean: Total,The Americas and the Caribbean: North America: Total,The Americas and the Caribbean: North America: United States,The Americas and the Caribbean: North America: Other North America,The Americas and the Caribbean: Central America,The Americas and the Caribbean: South America,The Americas and the Caribbean: The Caribbean: Total,The Americas and the Caribbean: The Caribbean: Jamaica,The Americas and the Caribbean: The Caribbean: Other Caribbean,Antarctica and Oceania: Total,Antarctica and Oceania: Antarctica,Antarctica and Oceania: Australasia: Total,Antarctica and Oceania: Australasia: Australia,Antarctica and Oceania: Australasia: Other Australasia,Antarctica and Oceania: Other Oceania,Other
WD22CD_proposed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
E05013714,10627,8340,7539,7408,34,69,23,0,5,0,2,0,0,128,671,478,233,64,20,46,23,18,62,245,48,85,17

Reference geographies
--

In [ ]:
# For filtering of source data
reference_names = ['United Kingdom', 'Great Britain', 'England and Wales',
                   'England', 'London', 'Lewisham']
years = [2011] # Just a precaution, in case any of the downloads include multiple periods

for datname in ['country_of_birth']:
  IPython.display.display(f"=== {datname} ===")
  # Load the Nomis data
  d = pd.read_csv(f"{output_dir}/census11_{datname}_references.csv")

  # Simplify the column names
  d.columns = d.columns.str.replace(
      # Remove the redundant title prefix and qualifier suffix
      r'^.*?: (.*?); measures: Value', 
      r'\1')
  d = d.rename(columns={'All categories: Country of birth': 'Total'})

  # Filter & transform to index data
  ref = filter_nomis_data(d, years, reference_names).drop(columns=['date', 'Rural Urban'])
  ref = ref.rename(columns={'geography': 'Name', 'geography code': 'Code'})
  ref.to_csv(f"{output_dir}/references_{datname}.csv", index=False)
  IPython.display.display(ref.head())

'=== country_of_birth ==='

,Name,Code,Total,Europe: Total,Europe: United Kingdom: Total,Europe: United Kingdom: England,Europe: United Kingdom: Northern Ireland,Europe: United Kingdom: Scotland,Europe: United Kingdom: Wales,Europe: Great Britain not otherwise specified,Europe: United Kingdom not otherwise specified,Europe: Guernsey,Europe: Jersey,Europe: Channel Islands not otherwise specified,Europe: Isle of Man,Europe: Ireland,Europe: Other Europe: Total,Europe: Other Europe: EU Countries: Total,Europe: Other Europe: EU countries: Member countries in March 2001: Total,Europe: Other Europe: EU countries: Member countries in March 2001: France,Europe: Other Europe: EU countries: Member countries in March 2001: Germany,Europe: Other Europe: EU countries: Member countries in March 2001: Italy,Europe: Other Europe: EU countries: Member countries in March 2001: Portugal,Europe: Other Europe: EU countries: Member countries in March 2001: Spain (including Canary Islands),Europe: Other Europe: EU countries: Member countries in March 2001: Other member countries in March 2001,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Total,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Lithuania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Poland,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Romania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Other EU accession countries,Europe: Other Europe: Rest of Europe: Total,Europe: Other Europe: Rest of Europe: Turkey,Europe: Other Europe: Rest of Europe: Other Europe,Africa: Total,Africa: North Africa,Africa: Central and Western Africa: Total,Africa: Central and Western Africa: Ghana,Africa: Central and Western Africa: Nigeria,Africa: Central and Western Africa: Other Central and Western Africa,Africa: South and Eastern Africa: Total,Africa: South and Eastern Africa: Kenya,Africa: South and Eastern Africa: Somalia,Africa: South and Eastern Africa: South Africa,Africa: South and Eastern Africa: Zimbabwe,Africa: South and Eastern Africa: Other South and Eastern Africa,Africa: Africa not otherwise specified,Middle East and Asia: Total,Middle East and Asia: Middle East: Total,Middle East and Asia: Middle East: Iran,Middle East and Asia: Middle East: Other Middle East,Middle East and Asia: Eastern Asia: Total,Middle East and Asia: Eastern Asia: China,Middle East and Asia: Eastern Asia: Hong Kong (Special Administrative Region of China),Middle East and Asia: Eastern Asia: Other Eastern Asia,Middle East and Asia: Southern Asia: Total,Middle East and Asia: Southern Asia: Bangladesh,Middle East and Asia: Southern Asia: India,Middle East and Asia: Southern Asia: Pakistan,Middle East and Asia: Southern Asia: Sri Lanka,Middle East and Asia: Southern Asia: Other Southern Asia,Middle East and Asia: South-East Asia: Total,Middle East and Asia: South-East Asia: Philippines,Middle East and Asia: South-East Asia: Other South-East Asia,Middle East and Asia: Central Asia,The Americas and the Caribbean: Total,The Americas and the Caribbean: North America: Total,The Americas and the Caribbean: North America: United States,The Americas and the Caribbean: North America: Other North America,The Americas and the Caribbean: Central America,The Americas and the Caribbean: South America,The Americas and the Caribbean: The Caribbean: Total,The Americas and the Caribbean: The Caribbean: Jamaica,The Americas and the Caribbean: The Caribbean: Other Caribbean,Antarctica and Oceania: Total,Antarctica and Oceania: Antarctica,Antarctica and Oceania: Australasia: Total,Antarctica and Oceania: Australasia: Australia,Antarctica and Oceania: Australasia: Other Australasia,Antarctica and Oceania: Other Oceania,Other
0,England,E92000001,53012456,48350320,45675317,44246592,206735,708872,506619,1822,4677,6076,8705,1582,8290,395182,2255168,1980259,894908,127601,262356,131195,85845,77554,210357,10853

Derivatives
==

Country of birth (%)
--

In [ ]:
for geog, geog_colname in zip(['oa11', 'wd22'], ['OA11CD', 'WD22CD_proposed']):
  IPython.display.display(f"=== {geog} ===")

  cob = pd.read_csv(f"{output_dir}/lbl_country_of_birth_{geog}.csv")

  # Relative measures
  d = cob[[geog_colname]].copy()
  var_colnames = cob.drop(columns=[geog_colname, 'Total']).columns
  for colname in var_colnames: 
    d[colname] = cob[colname] * 100. / cob.Total
  
  d.to_csv(f"{output_dir}/lbl_country_of_birth_share_{geog}.csv", index=False)
  IPython.display.display(d.head())

  # Groups only
  d_groups = d[[geog_colname]].copy()
  d_groups['United Kingdom'] = d['Europe: United Kingdom: Total']
  d_groups['Europe'] = d['Europe: Ireland'] + d['Europe: Other Europe: Total']
  for region in ['Africa', 'Middle East and Asia', 
                 'The Americas and the Caribbean', 'Antarctica and Oceania']:
                 d_groups[region] = d[f'{region}: Total']
  d_groups['Other'] = 100. - d_groups.sum(axis=1)
  
  d_groups.to_csv(f"{output_dir}/lbl_country_of_birth_groups_share_{geog}.csv", index=False)
  IPython.display.display(d_groups.head())

'=== oa11 ==='

,OA11CD,Europe: Total,Europe: United Kingdom: Total,Europe: United Kingdom: England,Europe: United Kingdom: Northern Ireland,Europe: United Kingdom: Scotland,Europe: United Kingdom: Wales,Europe: Great Britain not otherwise specified,Europe: United Kingdom not otherwise specified,Europe: Guernsey,Europe: Jersey,Europe: Channel Islands not otherwise specified,Europe: Isle of Man,Europe: Ireland,Europe: Other Europe: Total,Europe: Other Europe: EU Countries: Total,Europe: Other Europe: EU countries: Member countries in March 2001: Total,Europe: Other Europe: EU countries: Member countries in March 2001: France,Europe: Other Europe: EU countries: Member countries in March 2001: Germany,Europe: Other Europe: EU countries: Member countries in March 2001: Italy,Europe: Other Europe: EU countries: Member countries in March 2001: Portugal,Europe: Other Europe: EU countries: Member countries in March 2001: Spain (including Canary Islands),Europe: Other Europe: EU countries: Member countries in March 2001: Other member countries in March 2001,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Total,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Lithuania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Poland,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Romania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Other EU accession countries,Europe: Other Europe: Rest of Europe: Total,Europe: Other Europe: Rest of Europe: Turkey,Europe: Other Europe: Rest of Europe: Other Europe,Africa: Total,Africa: North Africa,Africa: Central and Western Africa: Total,Africa: Central and Western Africa: Ghana,Africa: Central and Western Africa: Nigeria,Africa: Central and Western Africa: Other Central and Western Africa,Africa: South and Eastern Africa: Total,Africa: South and Eastern Africa: Kenya,Africa: South and Eastern Africa: Somalia,Africa: South and Eastern Africa: South Africa,Africa: South and Eastern Africa: Zimbabwe,Africa: South and Eastern Africa: Other South and Eastern Africa,Africa: Africa not otherwise specified,Middle East and Asia: Total,Middle East and Asia: Middle East: Total,Middle East and Asia: Middle East: Iran,Middle East and Asia: Middle East: Other Middle East,Middle East and Asia: Eastern Asia: Total,Middle East and Asia: Eastern Asia: China,Middle East and Asia: Eastern Asia: Hong Kong (Special Administrative Region of China),Middle East and Asia: Eastern Asia: Other Eastern Asia,Middle East and Asia: Southern Asia: Total,Middle East and Asia: Southern Asia: Bangladesh,Middle East and Asia: Southern Asia: India,Middle East and Asia: Southern Asia: Pakistan,Middle East and Asia: Southern Asia: Sri Lanka,Middle East and Asia: Southern Asia: Other Southern Asia,Middle East and Asia: South-East Asia: Total,Middle East and Asia: South-East Asia: Philippines,Middle East and Asia: South-East Asia: Other South-East Asia,Middle East and Asia: Central Asia,The Americas and the Caribbean: Total,The Americas and the Caribbean: North America: Total,The Americas and the Caribbean: North America: United States,The Americas and the Caribbean: North America: Other North America,The Americas and the Caribbean: Central America,The Americas and the Caribbean: South America,The Americas and the Caribbean: The Caribbean: Total,The Americas and the Caribbean: The Caribbean: Jamaica,The Americas and the Caribbean: The Caribbean: Other Caribbean,Antarctica and Oceania: Total,Antarctica and Oceania: Antarctica,Antarctica and Oceania: Australasia: Total,Antarctica and Oceania: Australasia: Australia,Antarctica and Oceania: Australasia: Other Australasia,Antarctica and Oceania: Other Oceania,Other
0,E00016403,64.207650,55.737705,54.098361,0.273224,0.819672,0.546448,0.0,0.0,0.0,0.0,0.0,0.0,0.819672,7.650273,6.830601,2.459016,0.819672,0.000000,0.273224,0.819672,0.273224,0.273224,4.371585

,OA11CD,United Kingdom,Europe,Africa,Middle East and Asia,The Americas and the Caribbean,Antarctica and Oceania,Other
0,E00016403,55.737705,8.469945,21.311475,6.010929,7.650273,0.819672,0.000000e+00
1,E00016437,55.390335,10.408922,18.215613,11.338290,4.646840,0.000000,0.000000e+00
2,E00016439,46.630728,13.477089,24.258760,5.929919,8.894879,0.808625,-1.421085e-14
3,E00016442,51.977401,9.322034,26.271186,7.627119,4.237288,0.564972,0.000000e+00
4,E00016399,65.021459,7.725322,14.163090,7.939914,5.150215,0.000000,1.421085e-14


'=== wd22 ==='

,WD22CD_proposed,Europe: Total,Europe: United Kingdom: Total,Europe: United Kingdom: England,Europe: United Kingdom: Northern Ireland,Europe: United Kingdom: Scotland,Europe: United Kingdom: Wales,Europe: Great Britain not otherwise specified,Europe: United Kingdom not otherwise specified,Europe: Guernsey,Europe: Jersey,Europe: Channel Islands not otherwise specified,Europe: Isle of Man,Europe: Ireland,Europe: Other Europe: Total,Europe: Other Europe: EU Countries: Total,Europe: Other Europe: EU countries: Member countries in March 2001: Total,Europe: Other Europe: EU countries: Member countries in March 2001: France,Europe: Other Europe: EU countries: Member countries in March 2001: Germany,Europe: Other Europe: EU countries: Member countries in March 2001: Italy,Europe: Other Europe: EU countries: Member countries in March 2001: Portugal,Europe: Other Europe: EU countries: Member countries in March 2001: Spain (including Canary Islands),Europe: Other Europe: EU countries: Member countries in March 2001: Other member countries in March 2001,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Total,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Lithuania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Poland,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Romania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Other EU accession countries,Europe: Other Europe: Rest of Europe: Total,Europe: Other Europe: Rest of Europe: Turkey,Europe: Other Europe: Rest of Europe: Other Europe,Africa: Total,Africa: North Africa,Africa: Central and Western Africa: Total,Africa: Central and Western Africa: Ghana,Africa: Central and Western Africa: Nigeria,Africa: Central and Western Africa: Other Central and Western Africa,Africa: South and Eastern Africa: Total,Africa: South and Eastern Africa: Kenya,Africa: South and Eastern Africa: Somalia,Africa: South and Eastern Africa: South Africa,Africa: South and Eastern Africa: Zimbabwe,Africa: South and Eastern Africa: Other South and Eastern Africa,Africa: Africa not otherwise specified,Middle East and Asia: Total,Middle East and Asia: Middle East: Total,Middle East and Asia: Middle East: Iran,Middle East and Asia: Middle East: Other Middle East,Middle East and Asia: Eastern Asia: Total,Middle East and Asia: Eastern Asia: China,Middle East and Asia: Eastern Asia: Hong Kong (Special Administrative Region of China),Middle East and Asia: Eastern Asia: Other Eastern Asia,Middle East and Asia: Southern Asia: Total,Middle East and Asia: Southern Asia: Bangladesh,Middle East and Asia: Southern Asia: India,Middle East and Asia: Southern Asia: Pakistan,Middle East and Asia: Southern Asia: Sri Lanka,Middle East and Asia: Southern Asia: Other Southern Asia,Middle East and Asia: South-East Asia: Total,Middle East and Asia: South-East Asia: Philippines,Middle East and Asia: South-East Asia: Other South-East Asia,Middle East and Asia: Central Asia,The Americas and the Caribbean: Total,The Americas and the Caribbean: North America: Total,The Americas and the Caribbean: North America: United States,The Americas and the Caribbean: North America: Other North America,The Americas and the Caribbean: Central America,The Americas and the Caribbean: South America,The Americas and the Caribbean: The Caribbean: Total,The Americas and the Caribbean: The Caribbean: Jamaica,The Americas and the Caribbean: The Caribbean: Other Caribbean,Antarctica and Oceania: Total,Antarctica and Oceania: Antarctica,Antarctica and Oceania: Australasia: Total,Antarctica and Oceania: Australasia: Australia,Antarctica and Oceania: Australasia: Other Australasia,Antarctica and Oceania: Other Oceania,Other
0,E05013714,78.479345,70.941940,69.709231,0.319940,0.649290,0.216430,0.000000,0.047050,0.000000,0.018820,0.000000,0.000000,1.204479,6.314106,4.497977,2.192528,0.602240,0.188200,0.432

,WD22CD_proposed,United Kingdom,Europe,Africa,Middle East and Asia,The Americas and the Caribbean,Antarctica and Oceania,Other
0,E05013714,70.941940,7.518585,9.918133,4.629717,6.464665,0.508140,0.018820
1,E05013715,69.750386,10.936696,6.890118,6.060216,4.953680,1.338137,0.070767
2,E05013716,63.694460,12.732863,8.217279,6.631700,7.198408,1.452945,0.072346
3,E05013717,67.851152,9.405479,7.206312,6.920544,8.256197,0.285768,0.074548
4,E05013718,69.920332,10.919194,6.279708,4.773382,6.909018,1.164893,0.033474


In [ ]:
# For reference data
cob = pd.read_csv(f"{output_dir}/references_country_of_birth.csv")
geog_colname = 'Name'

# Relative measures
total = cob.Total
d = cob[[geog_colname]].copy()
var_colnames = cob.drop(columns=[geog_colname, 'Code', 'Total']).columns
for colname in var_colnames: 
  d[colname] = cob[colname] * 100. / total

d.to_csv(f"{output_dir}/references_country_of_birth_share.csv", index=False)
IPython.display.display(d.head())

# Groups only
d_groups = d[[geog_colname]].copy()
d_groups['United Kingdom'] = d['Europe: United Kingdom: Total']
d_groups['Europe'] = d['Europe: Ireland'] + d['Europe: Other Europe: Total']
for region in ['Africa', 'Middle East and Asia', 
                'The Americas and the Caribbean', 'Antarctica and Oceania']:
                d_groups[region] = d[f'{region}: Total']
d_groups['Other'] = 100. - d_groups.sum(axis=1)

d_groups.to_csv(f"{output_dir}/references_country_of_birth_groups_share.csv", index=False)
IPython.display.display(d_groups.head())

,Name,Europe: Total,Europe: United Kingdom: Total,Europe: United Kingdom: England,Europe: United Kingdom: Northern Ireland,Europe: United Kingdom: Scotland,Europe: United Kingdom: Wales,Europe: Great Britain not otherwise specified,Europe: United Kingdom not otherwise specified,Europe: Guernsey,Europe: Jersey,Europe: Channel Islands not otherwise specified,Europe: Isle of Man,Europe: Ireland,Europe: Other Europe: Total,Europe: Other Europe: EU Countries: Total,Europe: Other Europe: EU countries: Member countries in March 2001: Total,Europe: Other Europe: EU countries: Member countries in March 2001: France,Europe: Other Europe: EU countries: Member countries in March 2001: Germany,Europe: Other Europe: EU countries: Member countries in March 2001: Italy,Europe: Other Europe: EU countries: Member countries in March 2001: Portugal,Europe: Other Europe: EU countries: Member countries in March 2001: Spain (including Canary Islands),Europe: Other Europe: EU countries: Member countries in March 2001: Other member countries in March 2001,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Total,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Lithuania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Poland,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Romania,Europe: Other Europe: EU countries: Accession countries April 2001 to March 2011: Other EU accession countries,Europe: Other Europe: Rest of Europe: Total,Europe: Other Europe: Rest of Europe: Turkey,Europe: Other Europe: Rest of Europe: Other Europe,Africa: Total,Africa: North Africa,Africa: Central and Western Africa: Total,Africa: Central and Western Africa: Ghana,Africa: Central and Western Africa: Nigeria,Africa: Central and Western Africa: Other Central and Western Africa,Africa: South and Eastern Africa: Total,Africa: South and Eastern Africa: Kenya,Africa: South and Eastern Africa: Somalia,Africa: South and Eastern Africa: South Africa,Africa: South and Eastern Africa: Zimbabwe,Africa: South and Eastern Africa: Other South and Eastern Africa,Africa: Africa not otherwise specified,Middle East and Asia: Total,Middle East and Asia: Middle East: Total,Middle East and Asia: Middle East: Iran,Middle East and Asia: Middle East: Other Middle East,Middle East and Asia: Eastern Asia: Total,Middle East and Asia: Eastern Asia: China,Middle East and Asia: Eastern Asia: Hong Kong (Special Administrative Region of China),Middle East and Asia: Eastern Asia: Other Eastern Asia,Middle East and Asia: Southern Asia: Total,Middle East and Asia: Southern Asia: Bangladesh,Middle East and Asia: Southern Asia: India,Middle East and Asia: Southern Asia: Pakistan,Middle East and Asia: Southern Asia: Sri Lanka,Middle East and Asia: Southern Asia: Other Southern Asia,Middle East and Asia: South-East Asia: Total,Middle East and Asia: South-East Asia: Philippines,Middle East and Asia: South-East Asia: Other South-East Asia,Middle East and Asia: Central Asia,The Americas and the Caribbean: Total,The Americas and the Caribbean: North America: Total,The Americas and the Caribbean: North America: United States,The Americas and the Caribbean: North America: Other North America,The Americas and the Caribbean: Central America,The Americas and the Caribbean: South America,The Americas and the Caribbean: The Caribbean: Total,The Americas and the Caribbean: The Caribbean: Jamaica,The Americas and the Caribbean: The Caribbean: Other Caribbean,Antarctica and Oceania: Total,Antarctica and Oceania: Antarctica,Antarctica and Oceania: Australasia: Total,Antarctica and Oceania: Australasia: Australia,Antarctica and Oceania: Australasia: Other Australasia,Antarctica and Oceania: Other Oceania,Other
0,England,91.205584,86.159594,83.464520,0.389974,1.337180,0.955660,0.003437,0.008822,0.011461,0.016421,0.002984,0.015638,0.745451,4.254034,3.735460,1.688109,0.240700,0.494895,0.247480,0.161934,

,Name,United Kingdom,Europe,Africa,Middle East and Asia,The Americas and the Caribbean,Antarctica and Oceania,Other
0,England,86.159594,4.999485,2.434543,4.770835,1.250821,0.338034,0.046687
1,London,63.319236,12.178801,7.604814,11.830156,3.991710,1.035743,0.039540
2,Lewisham,66.259130,10.648640,9.162151,6.432753,6.692644,0.765174,0.039509


Results
--

In [ ]:
!ls -lh '{output_dir}'

total 6.4M
-rw------- 1 root root 5.2M Nov 29 12:06 census11_country_of_birth_oa11_london.csv
-rw------- 1 root root 8.1K Nov 29 12:04 census11_country_of_birth_references.csv
-rw------- 1 root root 111K Dec  7 11:33 lbl_country_of_birth_groups_share_oa11.csv
-rw------- 1 root root 2.8K Dec  7 11:33 lbl_country_of_birth_groups_share_wd22.csv
-rw------- 1 root root  40K Dec  7 11:32 lbl_country_of_birth_lsoa11.csv
-rw------- 1 root root 164K Dec  7 11:32 lbl_country_of_birth_oa11.csv
-rw------- 1 root root 882K Dec  7 11:33 lbl_country_of_birth_share_oa11.csv
-rw------- 1 root root  30K Dec  7 11:33 lbl_country_of_birth_share_wd22.csv
-rw------- 1 root root 9.0K Dec  7 11:32 lbl_country_of_birth_wd22.csv
drwx------ 2 root root 4.0K Nov 29 13:59 maps
-rw------- 1 root root 5.2K Dec  7 11:32 references_country_of_birth.csv
-rw------- 1 root root  531 Dec  7 11:38 references_country_of_birth_groups_share.csv
-rw------- 1 root root 8.1K Dec  7 11:38 references_country_of_birth_share.csv
